In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

In [164]:
tdf = df2#.columns
tdf['mood_r'] = tdf['mood_median'].rolling(5,min_periods=3).median()
tdf['mood_r'] = tdf['mood_r'].apply(lambda x: int(x) if not np.isnan(x) else np.nan)
tdf['arousal_r'] = tdf['circumplex.arousal_median'].rolling(5,min_periods=3).median()
tdf['valence_r'] = tdf['circumplex.valence_median'].rolling(5,min_periods=3).median()
tdf['activity_r'] = tdf['activity_mean'].rolling(5,min_periods=3).median()
tdf['screen_r'] = tdf['screen_sum'].rolling(5,min_periods=3).mean()
tdf['call_r'] = tdf['call_sum'].rolling(5,min_periods=3).mean()
tdf['sms_r'] = tdf['sms_sum'].rolling(5,min_periods=3).mean()
tdf['builtin_r'] = tdf['appCat.builtin_sum'].rolling(5,min_periods=3).mean()
tdf['comm_r'] = tdf['appCat.communication_sum'].rolling(5,min_periods=3).mean()
tdf['entertain_r'] = tdf['appCat.entertainment_sum'].rolling(5,min_periods=3).mean()
tdf['finance_r'] = tdf['appCat.finance_sum'].rolling(5,min_periods=3).mean()
tdf['game_r'] = tdf['appCat.game_sum'].rolling(5,min_periods=3).mean()
tdf['office_r'] = tdf['appCat.office_sum'].rolling(5,min_periods=3).mean()
tdf['other_r'] = tdf['appCat.other_sum'].rolling(5,min_periods=3).mean()
tdf['social_r'] = tdf['appCat.social_sum'].rolling(5,min_periods=3).mean()
tdf['travel_r'] = tdf['appCat.travel_sum'].rolling(5,min_periods=3).mean()
tdf['unknown_r'] = tdf['appCat.unknown_sum'].rolling(5,min_periods=3).mean()
tdf['utilities_r'] = tdf['appCat.utilities_sum'].rolling(5,min_periods=3).mean()
tdf['weather_r'] = tdf['appCat.weather_sum'].rolling(5,min_periods=3).mean()

tdf['target'] = tdf['mood_median'].shift(periods=-1)
tdf['target'] = tdf['target'].apply(lambda x: int(x) if not np.isnan(x) else np.nan)
tdf['target'] = tdf['target'].astype(str)
tdf = tdf.groupby('id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)
tdf = tdf.drop(['id','date','mood_median','circumplex.arousal_median',
       'circumplex.valence_median', 'activity_mean', 'screen_sum', 'call_sum',
       'sms_sum', 'appCat.builtin_sum', 'appCat.communication_sum',
       'appCat.entertainment_sum', 'appCat.finance_sum', 'appCat.game_sum',
       'appCat.office_sum', 'appCat.other_sum', 'appCat.social_sum',
       'appCat.travel_sum', 'appCat.unknown_sum', 'appCat.utilities_sum',
       'appCat.weather_sum'],axis=1)
tdf = tdf.dropna(subset=['mood_r', 'target','arousal_r', 'activity_r'])
#[['mood_r', 'target']]'']
#'appCat.social_sum','app_social_r','screen_sum','screen_r']]
#tdf['mood_r'] = tdf['mood_r'].dropna()
tdf

,mood_r,arousal_r,valence_r,activity_r,screen_r,call_r,sms_r,builtin_r,comm_r,entertain_r,finance_r,game_r,office_r,other_r,social_r,travel_r,unknown_r,utilities_r,weather_r,target
28,7.0,0.50,1.0,0.112144,4720.113400,2.2,0.2,1081.1744,3385.0096,309.1964,22.8046,0.0,34.4412,84.4124,1331.3404,190.5500,9.0346,99.2724,6.0772,6.0
29,6.0,0.75,0.5,0.081548,7143.315000,4.0,0.2,1254.5656,4728.4062,504.5906,29.6258,0.0,35.0432,97.7240,1976.0656,274.5110,9.0346,135.0188,6.0772,6.0
30,6.0,1.00,0.5,0.098374,8754.519000,3.8,0.4,1411.3234,5775.5092,448.0606,38.2366,0.0,35.0432,131.2188,2198.4136,274.5110,47.0446,175.3826,6.0772,7.0
31,7.0,1.00,1.0,0.098374,8685.616800,2.6,0.4,1016.9792,6152.4960,428.6652,38.7940,0.0,0.6020,102.7682,1994.9296,91.4220,47.0446,116.5394,6.0772,7.0
32,7.0,1.00,1.0,0.085703,8992.710800,2.4,0.4,1116.5588,6191.5852,537.8682,43.0226,0.0,37.0922,94.8460,2094.0794,93.4238,47.0446,128.8210,6.0772,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,6.0,0.50,0.5,0.020587,5743.653800,3.8,1.0,1282.9482,1431.7132,662.7334,0.0000,0.0,26.1060,102.7572,3698.5124,0.0000,0.0000,21.1640,0.0000,6.0
1942,6.0,1.00,0.5,0.020587,5526.749200,2.8,1.4,1278.2724,1592.9506,633.5772,0.0000,0.0,26.1060,121.0458,3714.1280,0.0000,0.0000,30.6310,0.0000,8.0
1943,6.0,1.00,0.5,0.062900,6184.113999,4.6,1.4,1592.3524,1816.3222,550.4800,0.0000,0.0,97.6878,205.8524,3778.4210,0.0000,0.0000,36.7642,0.0000,7.0
1944,6.0,1.00,1.0,0.103301,4886.446199,5.6,1.6,1482.7738,1450.5608,351.8246,0.0000,0.0,97.6878,196.0066,3311.3004,0.1878,0.0000,37.4040,0.0000,6.0


In [416]:
df_train = pd.read_csv('C:\\Users\\tijn2\\Documents\\CS\\DMT\\agg_train_set.csv', index_col=0)
df_val = pd.read_csv('C:\\Users\\tijn2\\Documents\\CS\\DMT\\agg_val_set.csv', index_col=0)
df_test = pd.read_csv('C:\\Users\\tijn2\\Documents\\CS\\DMT\\agg_test_set.csv', index_col=0)
X_train = df_train.drop(['id','date','month','day','target_num', 'target_class'],axis=1)
Y_train = df_train['target_class']
X_val = df_val.drop(['id','date','month','day','target_num', 'target_class'],axis=1)
Y_val = df_val['target_class']
X_test = df_test.drop(['id','date','month','day','target_num', 'target_class'],axis=1)
Y_test = df_test['target_class']

In [274]:
baseline =  np.array([7] * len(Y_test))
#baseline
test_accuracy = accuracy_score(Y_test, baseline)
print(f"Test accuracy: {test_accuracy:.4f}")

Test accuracy: 0.5225


In [396]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer

rfc = RandomForestClassifier()

# Define the hyperparameter space
param_distributions = {'n_estimators': [5,10,50,100], 'max_depth': [2, 5, 10, 20], 
                       'min_samples_split': [2,5,10,15], 'min_samples_leaf': [1,5,10,15]}

rfc_random = RandomizedSearchCV(estimator=rfc, param_distributions=param_distributions, n_iter=10, cv=3)
rfc_random.fit(X_train, Y_train)

# Get the best hyperparameters
best_params = rfc_random.best_params_
print(best_params)

rfc_best = RandomForestClassifier(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], 
                                  min_samples_split=best_params['min_samples_split'],
                                  min_samples_leaf=best_params['min_samples_leaf'])

# Train the model on the training data
rfc_best.fit(X_train, Y_train)

importances = rfc_best.feature_importances_

# Print feature importances
for feature, importance in zip(X_train.columns,importances):
    print(f'{feature}: {importance:.4f}')
    
    
# Make predictions on the test data
#y_pred = rfc_best.predict(val_df.drop(columns=['target']))
#print("Accuracy on validation set: ", accuracy_score(val_df['target'], y_pred))

# Evaluate the decision tree on the test set
test_predictions = rfc_best.predict(X_test)
test_accuracy = accuracy_score(Y_test, test_predictions)
print(f"Test accuracy: {test_accuracy:.4f}")

{'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_depth': 20}
mood_mean: 0.1582
circumplex.arousal_median: 0.0394
circumplex.valence_median: 0.0319
activity_mean: 0.0583
screen_sum: 0.0710
call_sum: 0.0358
sms_sum: 0.0332
appCat.builtin_sum: 0.0596
appCat.communication_sum: 0.0673
appCat.entertainment_sum: 0.0841
appCat.finance_sum: 0.0246
appCat.game_sum: 0.0207
appCat.office_sum: 0.0342
appCat.other_sum: 0.0684
appCat.social_sum: 0.0780
appCat.travel_sum: 0.0577
appCat.unknown_sum: 0.0239
appCat.utilities_sum: 0.0376
appCat.weather_sum: 0.0158
Test accuracy: 0.6036


In [338]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(Y_test, test_predictions)

num_classes = confusion_matrix.shape[0]
class_labels = np.arange(num_classes)
print("Confusion matrix:")
print("".join(["{:8d}".format(i) for i in class_labels]))
print("-" * 8 * num_classes)
for i in range(num_classes):
    print("".join(["{:8d}".format(confusion_matrix[i][j]) for j in range(num_classes)]))

Confusion matrix:
       0       1       2       3       4
----------------------------------------
       0       1       0       0       0
       0       7      12       0       0
       0       3      48       7       0
       0       0      16      14       0
       0       0       0       3       0


# Regression

In [421]:
#X = rtdf.drop(columns=['target'])
#Y = rtdf['target']

#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25,random_state=42)
#train_df, test_df = train_test_split(rtdf, test_size=0.25)#, random_state=42)
#train_df, val_df = train_test_split(train_df, test_size=0.2)#, random_state=42)
#print("Training set size:", len(train_df))
#print("Validation set size:", len(val_df))
#print("Test set size:", len(test_df))
X_train = df_train.drop(['id','date','target_num','month', 'day','target_class'],axis=1)
Y_train = df_train['target_num']
X_val = df_val.drop(['id','date','target_num','month', 'day','target_class'],axis=1)
Y_val = df_val['target_num']
X_test = df_test.drop(['id','date','target_num','month', 'day', 'target_class'],axis=1)
Y_test = df_test['target_num']

In [434]:
from sklearn.metrics import mean_absolute_error
# Create a linear regression model
model = LinearRegression()

mse_list = []
mae_list = []
ints = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
for i in range(1,len(X_train.columns)): 
    
    # Create the RFE object and set the number of features to select
    rfe = RFE(model, n_features_to_select=i)

    # Fit the RFE model to the training data
    rfe.fit(X_train, Y_train)

    # Print the selected features
    #print("Selected Features:")
    #for i in range(X_train.shape[1]):
    #    if rfe.support_[i]:
    #        print(X_train.columns[i])
    # Select the RFE selected features
    X_train_rfe = rfe.transform(X_val)
    X_test_rfe = rfe.transform(X_test)

    # Train a linear regression model on the RFE selected features
    model = LinearRegression()
    model.fit(X_train_rfe, Y_val)
    val_pred = model.predict(X_train_rfe)
    #print(mean_squared_error(Y_val, val_pred))
    #print(model.coef_)
    #print(model.intercept_)
    # Predict on the test set and evaluate the model
    y_pred = model.predict(X_test_rfe)
    mse = mean_squared_error(Y_test, y_pred)
    mae = mean_absolute_error(Y_test, y_pred)
    mse_list.append(mse)
    mae_list.append(mae)
    print('amount of features:', i, "MSE on test set:", mse)
    print(mean_absolute_error(Y_test, y_pred))
    print(mse+mae)

amount of features: 1 MSE on test set: 0.2738775096100233
0.4084264279124175
0.6823039375224408
amount of features: 2 MSE on test set: 0.2744024426063269
0.4085084989262962
0.6829109415326231
amount of features: 3 MSE on test set: 0.2737465687991398
0.40945640469658295
0.6832029734957228
amount of features: 4 MSE on test set: 0.2733150329299036
0.408816471685317
0.6821315046152205
amount of features: 5 MSE on test set: 0.27367002632114573
0.40794816063393896
0.6816181869550847
amount of features: 6 MSE on test set: 0.2737190904054495
0.408110889544651
0.6818299799501005
amount of features: 7 MSE on test set: 0.2723687756958501
0.40705752383178734
0.6794262995276374
amount of features: 8 MSE on test set: 0.28212483889505974
0.4261553882399316
0.7082802271349913
amount of features: 9 MSE on test set: 0.2817559952349471
0.4252548303473134
0.7070108255822605
amount of features: 10 MSE on test set: 0.29331820105373113
0.43428992394301486
0.727608124996746
amount of features: 11 MSE on test 

In [407]:
#df_test['pred'] = y_pred
#df_train['pred'] = 

In [409]:
#df_test.to_csv('C:\\Users\\tijn2\\Documents\\CS\\DMT\\met_pred.csv')

In [285]:
# Select the RFE selected features
#X_train_rfe = rfe.transform(X_train)
#X_test_rfe = rfe.transform(X_test)

# Train a linear regression model on the RFE selected features
#model = LinearRegression()
#model.fit(X_train_rfe, Y_train)

# Predict on the test set and evaluate the model
#y_pred = model.predict(X_test_rfe)
#mse = mean_squared_error(Y_test, y_pred)
#print("MSE on test set:", mse)

#from sklearn.metrics import mean_absolute_error
#mean_absolute_error(Y_test, y_pred)

MSE on test set: 0.27319030101810143


0.41034394778704086